## 분석의 목적 :각나라의 디즈니랜드의 평가가 어떤지 알아보고 부정의 리뷰를 통해 고객들의 니즈가 무엇인지 파악하여 개선방법을 찾아보자

In [10]:
pip install spacy

  Using cached pydantic-1.8.2-cp37-cp37m-win_amd64.whl (1.9 MB)
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: requests<3.0.0,>=2.13.0 in c:\python37\lib\site-packages (from spacy) (2.25.0)



You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


In [1]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import spacy
import re

from spacy.tokenizer import Tokenizer

In [2]:
# 데이터 로드
df = pd.read_csv('Raw_data.csv',encoding='latin1')
df

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong
...,...,...,...,...,...,...
42651,1765031,5,missing,United Kingdom,i went to disneyland paris in july 03 and thou...,Disneyland_Paris
42652,1659553,5,missing,Canada,2 adults and 1 child of 11 visited Disneyland ...,Disneyland_Paris
42653,1645894,5,missing,South Africa,My eleven year old daughter and myself went to...,Disneyland_Paris
42654,1618637,4,missing,United States,"This hotel, part of the Disneyland Paris compl...",Disneyland_Paris


In [22]:
# 데이터 탐색
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42656 entries, 0 to 42655
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Review_ID          42656 non-null  int64 
 1   Rating             42656 non-null  int64 
 2   Year_Month         42656 non-null  object
 3   Reviewer_Location  42656 non-null  object
 4   Review_Text        42656 non-null  object
 5   Branch             42656 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.0+ MB


In [23]:
# 결측값 확인
df.isnull().sum()

Review_ID            0
Rating               0
Year_Month           0
Reviewer_Location    0
Review_Text          0
Branch               0
dtype: int64

In [3]:
# 필요없는 컬럼 제거
df = df.drop(columns = ['Review_ID', 'Year_Month'])
df

,Rating,Reviewer_Location,Review_Text,Branch
0,4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,4,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong
...,...,...,...,...
42651,5,United Kingdom,i went to disneyland paris in july 03 and thou...,Disneyland_Paris
42652,5,Canada,2 adults and 1 child of 11 visited Disneyland ...,Disneyland_Paris
42653,5,South Africa,My eleven year old daughter and myself went to...,Disneyland_Paris
42654,4,United States,"This hotel, part of the Disneyland Paris compl...",Disneyland_Paris


In [25]:
# 국가의 종류 확인
df['Branch'].unique()

array(['Disneyland_HongKong', 'Disneyland_California', 'Disneyland_Paris'],
      dtype=object)

In [4]:
# 리뷰 컬럼 확인
df['Review_Text']

0        If you've ever been to Disneyland anywhere you...
1        Its been a while since d last time we visit HK...
2        Thanks God it wasn   t too hot or too humid wh...
3        HK Disneyland is a great compact park. Unfortu...
4        the location is not in the city, took around 1...
                               ...                        
42651    i went to disneyland paris in july 03 and thou...
42652    2 adults and 1 child of 11 visited Disneyland ...
42653    My eleven year old daughter and myself went to...
42654    This hotel, part of the Disneyland Paris compl...
42655    I went to the Disneyparis resort, in 1996, wit...
Name: Review_Text, Length: 42656, dtype: object

In [16]:
# 언어모델 다운받기
!python -m spacy download en

You should consider upgrading via the 'C:\Python37\python.exe -m pip install --upgrade pip' command.


[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
nlp = spacy.load("en_core_web_sm")

tokenizer = Tokenizer(nlp.vocab)

In [7]:
# 토큰화를 위한 파이프라인 구성
 
tokens = []

for doc in tokenizer.pipe(df['Review_Text']):
    doc_tokens = [re.sub(r"[^a-z0-9]", "", token.text.lower()) for token in doc]
    tokens.append(doc_tokens)
    
df['Review_Text'] = tokens
df['Review_Text'].head()

0    [if, youve, ever, been, to, disneyland, anywhe...
1    [its, been, a, while, since, d, last, time, we...
2    [thanks, god, it, wasn, , t, too, hot, or, too...
3    [hk, disneyland, is, a, great, compact, park, ...
4    [the, location, is, not, in, the, city, took, ...
Name: Review_Text, dtype: object

In [8]:
df

,Rating,Reviewer_Location,Review_Text,Branch
0,4,Australia,"[if, youve, ever, been, to, disneyland, anywhe...",Disneyland_HongKong
1,4,Philippines,"[its, been, a, while, since, d, last, time, we...",Disneyland_HongKong
2,4,United Arab Emirates,"[thanks, god, it, wasn, , t, too, hot, or, too...",Disneyland_HongKong
3,4,Australia,"[hk, disneyland, is, a, great, compact, park, ...",Disneyland_HongKong
4,4,United Kingdom,"[the, location, is, not, in, the, city, took, ...",Disneyland_HongKong
...,...,...,...,...
42651,5,United Kingdom,"[i, went, to, disneyland, paris, in, july, 03,...",Disneyland_Paris
42652,5,Canada,"[2, adults, and, 1, child, of, 11, visited, di...",Disneyland_Paris
42653,5,South Africa,"[my, eleven, year, old, daughter, and, myself,...",Disneyland_Paris
42654,4,United States,"[this, hotel, part, of, the, disneyland, paris...",Disneyland_Paris
